## Chieftains of the Northwind

In [1]:
%pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


### Section 1: prepare data

In [59]:
import pandas as pd

In [122]:
#Collect all the seperate datasets
df_adhd = pd.read_csv('data/adhd.csv')
df_depression  = pd.read_csv('data/depression.csv')
df_ocd = pd.read_csv('data/ocd.csv')

#Create a classification column for the training model thats ordered on occurance frequency
df_ocd['classification'] = 0
df_adhd['classification'] = 1
df_depression['classification'] = 2

#Compile them all together in one dataframe
df_compiled = pd.concat((df_ocd, df_adhd, df_depression))
df_compiled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103966 entries, 0 to 24030
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   author          103966 non-null  object 
 1   body            103223 non-null  object 
 2   created_utc     103966 non-null  object 
 3   id              103966 non-null  object 
 4   num_comments    103966 non-null  int64  
 5   score           103966 non-null  int64  
 6   subreddit       103966 non-null  object 
 7   title           103966 non-null  object 
 8   upvote_ratio    103966 non-null  float64
 9   url             103966 non-null  object 
 10  classification  103966 non-null  int64  
dtypes: float64(1), int64(3), object(7)
memory usage: 9.5+ MB


In [123]:
df_compiled

,author,body,created_utc,id,num_comments,score,subreddit,title,upvote_ratio,url,classification
0,christmascamearly,"So I know the title/ post sounds gross, but I ...",2021-12-22T18:26:02.000Z,rmbecy,1,1,OCD,OCD with Bathroom use/ Do I have OCD?,1.00,https://www.reddit.com/r/OCD/comments/rmbecy/o...,0
1,Yupitisme1995,"Sorry, I've asked before, but I'm in a dark da...",2021-12-22T18:08:52.000Z,rmb0ql,2,1,OCD,Is there anyone I can talk to? I'm desperate.,1.00,https://www.reddit.com/r/OCD/comments/rmb0ql/i...,0
2,moonwalker04,I changed my account password twice today. The...,2021-12-22T17:59:53.000Z,rmat68,0,1,OCD,HAVE I BEEN HACKED? PLEASE HELP,1.00,https://www.reddit.com/r/OCD/comments/rmat68/h...,0
3,terrapin09,I have ADHD and I highly suspect I have OCD bu...,2021-12-22T17:55:59.000Z,rmaq5r,0,1,OCD,ADHD and OCD,1.00,https://www.reddit.com/r/OCD/comments/rmaq5r/a...,0
4,loveness7,[removed],2021-12-22T17:52:04.000Z,rman3v,0,1,OCD,IS IT STILL A FALSE MEMORY IF YOU HAVE THE IMA...,1.00,https://www.reddit.com/r/OCD/comments/rman3v/i...,0
...,...,...,...,...,...,...,...,...,...,...,...
24026,[deleted],[deleted],2021-11-06T21:05:57.000Z,qo9a1n,0,2,depression,I don't know how to stop this,1.00,https://www.reddit.com/r/depression/comments/q...,2
24027,[deleted],[removed],2021-11-06T21:02:26.000Z,qo97h7,0,1,depression,i have never felt like this,1.00,https://www.reddit.com/r/depression/comments/q...,2
24028,[deleted],[deleted],2021-11-06T21:02:24.000Z,qo97gn,1,8,depression,My dad thinks I’m not mentally ill enough,1.00,https://www.reddit.com/r/depression/comments/q...,2
24029,[deleted],[deleted],2021-11-06T21:02:21.000Z,qo97fo,0,2,depression,i feel like nobody gives a shit about me,1.00,https://www.reddit.com/r/depression/comments/q...,2


In [124]:
df_compiled.dropna(inplace=True)
df_compiled = df_compiled[df_compiled.body != '[deleted]']
df_compiled = df_compiled[df_compiled.body != '[removed]']
df_compiled.info()


<class 'pandas.core.frame.DataFrame'>
Index: 59928 entries, 0 to 24023
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   author          59928 non-null  object 
 1   body            59928 non-null  object 
 2   created_utc     59928 non-null  object 
 3   id              59928 non-null  object 
 4   num_comments    59928 non-null  int64  
 5   score           59928 non-null  int64  
 6   subreddit       59928 non-null  object 
 7   title           59928 non-null  object 
 8   upvote_ratio    59928 non-null  float64
 9   url             59928 non-null  object 
 10  classification  59928 non-null  int64  
dtypes: float64(1), int64(3), object(7)
memory usage: 5.5+ MB


That removed 40k rows! 

In [125]:
#NAN row checking
nan_rows = df_compiled[df_compiled.isnull().T.any().T]
nan_rows

,author,body,created_utc,id,num_comments,score,subreddit,title,upvote_ratio,url,classification


In [115]:
# df_compiled[df_compiled['score'] <= 1].count()

In [126]:
#Drop all columns unnessesary for our model
df_compiled['text'] = (df_compiled['title'] + '\n' + df_compiled['body']).astype('string')
df_compiled.drop(df_compiled.columns.difference(['text','classification']), axis=1, inplace=True)

#Ensure columns are correct types
df_compiled.info()


<class 'pandas.core.frame.DataFrame'>
Index: 59928 entries, 0 to 24023
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   classification  59928 non-null  int64 
 1   text            59928 non-null  string
dtypes: int64(1), string(1)
memory usage: 1.4 MB


In [127]:
#Upload CSV
df_compiled.to_csv('input_refined.csv')